In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
from math import radians, sin, cos, sqrt, asin
from bokeh.plotting import figure, show, output_notebook
from bokeh.resources import INLINE

In [3]:
output_notebook(resources=INLINE)

In [4]:
df = pd.read_csv("/content/dataset.csv")

# Convert timestamp (day-first)
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)

# Preprocessing for demand and competitive models
traffic_map = {'low': 1, 'medium': 2, 'high': 3}
df['TrafficWeight'] = df['TrafficConditionNearby'].map(traffic_map)

vehicle_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_map)

df = df.sort_values(by=['SystemCodeNumber', 'Timestamp']).copy()

In [5]:
def model_1(df, alpha=5.0, base_price=10.0):
    all_prices = {}
    for lot, group in df.groupby('SystemCodeNumber'):
        group = group.sort_values('Timestamp')
        prices = []
        p = base_price
        for _, row in group.iterrows():
            occ = row['Occupancy']
            cap = row['Capacity']
            p = p + alpha * (occ / cap)
            prices.append(p)
        all_prices[lot] = {'Timestamp': group['Timestamp'].values, 'Price': prices}
    return all_prices


In [6]:
def model_2(df,
            alpha=0.6, beta=0.2, gamma=0.3,
            delta=0.5, epsilon=0.4, lamb=0.5,
            base_price=10.0):
    all_prices = {}
    for lot, group in df.groupby('SystemCodeNumber'):
        group = group.sort_values('Timestamp')
        prices = []
        for _, row in group.iterrows():
            occ = row['Occupancy']
            cap = row['Capacity']
            queue = row['QueueLength']
            traffic = row['TrafficWeight']
            special = row['IsSpecialDay']
            vehicle = row['VehicleWeight']
            demand = (alpha * (occ / cap) + beta * queue - gamma * traffic + delta * special + epsilon * vehicle)
            norm_demand = max(0.0, min(demand / 10.0, 1.0))
            price = base_price * (1 + lamb * norm_demand)
            price = max(base_price * 0.5, min(price, base_price * 2))
            prices.append(price)
        all_prices[lot] = {'Timestamp': group['Timestamp'].values, 'Price': prices}
    return all_prices


In [7]:
def haversine(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return 6371 * c  # KM

def model_3(df,
            alpha=0.6, beta=0.2, gamma=0.3,
            delta=0.5, epsilon=0.4, lamb=0.5,
            base_price=10.0):
    location_df = df.groupby("SystemCodeNumber")[['Latitude', 'Longitude']].first()
    all_prices = {}
    last_prices = {lot: base_price for lot in location_df.index}

    for lot_id, group in df.groupby('SystemCodeNumber'):
        group = group.sort_values('Timestamp')
        prices = []
        for _, row in group.iterrows():
            occ = row['Occupancy']
            cap = row['Capacity']
            queue = row['QueueLength']
            traffic = row['TrafficWeight']
            special = row['IsSpecialDay']
            vehicle = row['VehicleWeight']
            demand = (alpha * (occ / cap) + beta * queue - gamma * traffic + delta * special + epsilon * vehicle)
            norm_demand = max(0.0, min(demand / 10.0, 1.0))
            price = base_price * (1 + lamb * norm_demand)
            price = max(base_price * 0.5, min(price, base_price * 2))
            lat1, lon1 = location_df.loc[lot_id]
            adjustment = 0
            for other_id in location_df.index:
                if other_id == lot_id:
                    continue
                lat2, lon2 = location_df.loc[other_id]
                distance = haversine(lat1, lon1, lat2, lon2)
                if distance <= 1.0:
                    competitor_price = last_prices[other_id]
                    if competitor_price < price:
                        adjustment -= 0.25
                    else:
                        adjustment += 0.25
            price = price + adjustment
            price = max(base_price * 0.5, min(price, base_price * 2))
            prices.append(price)
            last_prices[lot_id] = price
        all_prices[lot_id] = {'Timestamp': group['Timestamp'].values, 'Price': prices}
    return all_prices


In [8]:
model1_prices = model_1(df)
model2_prices = model_2(df)
model3_prices = model_3(df)

def plot_all_models(lot_id):
    time1 = model1_prices[lot_id]['Timestamp']
    p = figure(title=f"All Models for Parking Lot: {lot_id}", x_axis_type='datetime', width=900, height=400)
    p.line(time1, model1_prices[lot_id]['Price'], color='blue', legend_label='Model 1: Linear')
    p.line(time1, model2_prices[lot_id]['Price'], color='green', legend_label='Model 2: Demand')
    p.line(time1, model3_prices[lot_id]['Price'], color='red', legend_label='Model 3: Competitive')
    p.legend.location = 'top_left'
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Price ($)'
    show(p)

# Example usage
plot_all_models("BHMBCCMKT01")

Output hidden; open in https://colab.research.google.com to view.

In [9]:
model1_prices
model2_prices
model3_prices

{'BHMBCCMKT01': {'Timestamp': array(['2016-10-04T07:59:00.000000000', '2016-10-04T08:25:00.000000000',
         '2016-10-04T08:59:00.000000000', ...,
         '2016-12-19T15:29:00.000000000', '2016-12-19T16:03:00.000000000',
         '2016-12-19T16:30:00.000000000'], dtype='datetime64[ns]'),
  'Price': [7.181715771230502,
   7.183275563258231,
   7.29159445407279,
   7.305632582322357,
   7.227989601386483,
   7.442027729636049,
   7.563864818024264,
   13.0,
   13.0,
   7.588301559792029,
   7.589861351819758,
   7.586741767764298,
   7.373743500866549,
   13.0,
   13.0,
   7.3357885615251295,
   7.134228769497399,
   13.0,
   7.178076256499134,
   13.0,
   7.286915077989603,
   13.0,
   7.4092720970537265,
   7.316551126516465,
   13.0,
   13.0,
   7.442547660311959,
   7.545147313691508,
   7.540987868284228,
   7.543067590987869,
   13.0,
   7.332149046793761,
   13.0,
   7.317071057192376,
   7.312911611785097,
   7.109272097053726,
   7.180155979202773,
   7.182755632582323,
   7